# Personal Assitant for a Data Scientist

In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.1/289.1 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:
import pandas as pd
import numpy as np

#llm
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

#langchain

import langchain
from langchain.llms.base import LLM

from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"using: {device}")

using: cuda


In [ ]:
model_name = "bigscience/bloom-1b7"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
prompt = f"""
What is machine learning?
"""

In [ ]:
inputs = tokenizer(prompt, return_tensors ='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=20,
    )[0],
    skip_special_tokens = True
)

In [ ]:
output.split('\n')[2]

'Machine learning is a branch of artificial intelligence that uses computers to learn from data. It is a process'

## load the model to langchain

In [ ]:
class CustomLLM(LLM):
  def _call(self, prompt, stop=None, run_manager=None) -> str:
    inputs = tokenizer(prompt, return_tensors ='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=20,
        )[0],
        skip_special_tokens = True
    )
    return output


  @property
  def _llm_type(self) ->str:
    return "custom"


In [ ]:
llm = CustomLLM()

## Create a basic chain to prove the model - Sequential Chain

In [ ]:
first_prompt = ChatPromptTemplate.from_template(
    "Transale the following reviwe to english:"
    "\n\n{Review}"
)

In [ ]:
chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key = "English_Review")

In [ ]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)

In [ ]:
chain_two =   LLMChain(llm=llm, prompt=second_prompt,
                       output_key = "Summary")

In [ ]:
#overall chain
overall_chain = SequentialChain(
    chains = [chain_one, chain_two],
    input_variables=["Review"],
    output_variables = ["English_Review","Summary"],
    verbose = True
)

In [ ]:
review = f"""
Umienko es una historia de amor
"""

In [ ]:
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': '\nUmienko es una historia de amor\n',
 'English_Review': 'Human: Transale the following reviwe to english:\n\n\nUmienko es una historia de amor\nque se desarrolla en la Rusia de los años treinta.\nEl protagonista es un joven de familia noble,',
 'Summary': 'Human: Can you summarize the following review in 1 sentence:\n\nHuman: Transale the following reviwe to english:\n\n\nUmienko es una historia de amor\nque se desarrolla en la Rusia de los años treinta.\nEl protagonista es un joven de familia noble, que se enamora de una joven campesina.\nLa historia se desarrolla en un entorno rural, con'}

#create a Basic chain to prove the model - conversational chain with retriever

## Agent - langchain expresion language (lcel)

In [ ]:
#tool for python

In [ ]:
#tool for wikiperdia

In [ ]:
#tool for access to personal information